In [2]:
# توليد بيانات المبيعات الشهرية للمنتجات الأربعة وحفظها في initial.csv

import pandas as pd
from utils import generate_random_sales

# إنشاء تواريخ شهرية من 2025-01-01 إلى 2025-12-01
dates = pd.date_range(start='2025-01-01', end='2025-12-01', freq='MS')

# توليد بيانات عشوائية لكل منتج
product_a_sales = generate_random_sales(50, 100, 12)
product_b_sales = generate_random_sales(30, 80, 12)
product_c_sales = generate_random_sales(20, 60, 12)
product_d_sales = generate_random_sales(10, 50, 12)

# إنشاء DataFrame
df = pd.DataFrame({
    'Date': dates,
    'Product_A': product_a_sales,
    'Product_B': product_b_sales,
    'Product_C': product_c_sales,
    'Product_D': product_d_sales
})

# حفظ البيانات إلى ملف initial.csv داخل مجلد data/
df.to_csv('data/initial.csv', index=False)


In [ ]:
import pandas as pd

# تحميل البيانات الأولية
df = pd.read_csv('data/initial.csv')

# حساب مجموع المبيعات في كل شهر
df['Total_Sales'] = df[['Product_A', 'Product_B', 'Product_C', 'Product_D']].sum(axis=1)

# حساب متوسط المبيعات في كل شهر
df['Average_Sales'] = df[['Product_A', 'Product_B', 'Product_C', 'Product_D']].mean(axis=1)

# حساب نسبة النمو الشهري Total_Sales مقارنة بالشهر السابق
df['Month_over_Month_Growth'] = df['Total_Sales'].pct_change().fillna(0) * 100

# استخراج رقم الشهر من التاريخ
df['Month'] = pd.to_datetime(df['Date']).dt.month

# تعيين الربع "Quarter"
def assign_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

df['Quarter'] = df['Month'].apply(assign_quarter)

# تحديد المنتج الأعلى والأقل مبيعات في كل شهر
sales_cols = ['Product_A', 'Product_B', 'Product_C', 'Product_D']
df['Max_Sales_Product'] = df[sales_cols].idxmax(axis=1)
df['Min_Sales_Product'] = df[sales_cols].idxmin(axis=1)

# حفظ النتائج في final.csv داخل مجلد data
df.to_csv('data/final.csv', index=False)
